# The World Bank's international debt data

It's not that we humans only take debts to manage our necessities. A country may also take debt to manage its economy. For example, infrastructure spending is one costly ingredient required for a country's citizens to lead comfortable lives. The World Bank is the organization that provides debt to countries.

In this project, I am going to analyze international debt data collected by The World Bank. The dataset contains information about the amount of debt (in USD) owed by developing countries across several categories. I am going to find the answers to questions like:

* What is the total amount of debt that is owed by the countries listed in the dataset?

* Which country owns the maximum amount of debt and what does that amount look like?

* What is the average amount of debt owed by countries across different debt indicators?

In [1]:
import os
from dotenv import load_dotenv
import psycopg2
import pandas as pd


load_dotenv()

db_host = os.getenv("host")
db_name = os.getenv("database")
db_user=os.getenv("user")
db_password= os.getenv("password")
db_port= os.getenv("port")


conn = psycopg2.connect(
                host=db_host,
                database=db_name,
                user=db_user,
                password=db_password,
                port=db_port)
        
cursor = conn.cursor()

cursor.execute("SELECT * FROM international_debt LIMIT 5;")

columns = cursor.description 
result = [{columns[index][0]:column for index, column in enumerate(value)} for value in cursor.fetchall()]
pd.DataFrame(result)



,country_name,country_code,indicator_name,indicator_code,debt
0,Afghanistan,AFG,"Disbursements on external debt, long-term (DIS...",DT.DIS.DLXF.CD,72894453.700000003
1,Afghanistan,AFG,"Interest payments on external debt, long-term ...",DT.INT.DLXF.CD,53239440.100000001
2,Afghanistan,AFG,"PPG, bilateral (AMT, current US$)",DT.AMT.BLAT.CD,61739336.899999999
3,Afghanistan,AFG,"PPG, bilateral (DIS, current US$)",DT.DIS.BLAT.CD,49114729.399999999
4,Afghanistan,AFG,"PPG, bilateral (INT, current US$)",DT.INT.BLAT.CD,39903620.100000001


From the output, one can see the debt owed by Afghanistan in the different debt indicators. The number of different countries is not know yet. Although, there are repititions in the country names because a country is most likely to have debt in more than one debt indicator. Therefore I will make a count of unique countries. 

In [2]:
# Number of distinct countries
cursor = conn.cursor()
cursor.execute("SELECT COUNT(DISTINCT(country_name)) AS total_distinct_countries FROM international_debt;")
columns = cursor.description 
result = [{columns[index][0]:column for index, column in enumerate(value)} for value in cursor.fetchall()]
pd.DataFrame(result)

,total_distinct_countries
0,124


There are 124 unique countries present in the table. 

The column indicator_name briefly specifies the purpose of taking the debt, while indicator_code indicates the categories of these debts. Knowing the various debt indicators will help understand the areas in which a country can possibly be indebted to.

In [3]:
# Distinct debt indicators
cursor = conn.cursor()
cursor.execute("SELECT DISTINCT(indicator_code) AS distinct_debt_indicators FROM international_debt ORDER BY distinct_debt_indicators;")
columns = cursor.description 
result = [{columns[index][0]:column for index, column in enumerate(value)} for value in cursor.fetchall()]
pd.DataFrame(result)

,distinct_debt_indicators
0,DT.AMT.BLAT.CD
1,DT.AMT.DLXF.CD
2,DT.AMT.DPNG.CD
3,DT.AMT.MLAT.CD
4,DT.AMT.OFFT.CD
5,DT.AMT.PBND.CD
6,DT.AMT.PCBK.CD
7,DT.AMT.PROP.CD
8,DT.AMT.PRVT.CD
9,DT.DIS.BLAT.CD


To have a sense of how the overall economy of the entire world is holding up, I will calculate the total amount
of debt owed by different countries.

In [4]:
# Total amount of debt owed by different countries

cursor = conn.cursor()
cursor.execute("SELECT ROUND(SUM(debt)/1000000, 2) AS total_debt FROM international_debt;")
columns = cursor.description 
result = [{columns[index][0]:column for index, column in enumerate(value)} for value in cursor.fetchall()]
pd.DataFrame(result)

,total_debt
0,3079734.49


The total amount of debt is about 1 trillion. The next thing is to find the country that owns the highest amount of debt along with th amount. This will be the sum of different debts owed by a country across several categories.  

In [5]:
# Country with the highest amount of debt

cursor = conn.cursor()
cursor.execute("SELECT country_name, SUM(debt) AS total_debt FROM international_debt GROUP BY country_name ORDER BY total_debt DESC LIMIT 1;")
columns = cursor.description 
result = [{columns[index][0]:column for index, column in enumerate(value)} for value in cursor.fetchall()]
pd.DataFrame(result)

,country_name,total_debt
0,China,285793494734.200001568


China has the highest debt in the table. To have a better sense of the distribution of the amount of debt across different indicators, I will find out on an average how much debt a country owes.

In [6]:
# Average debt owed 

cursor = conn.cursor()
cursor.execute("SELECT indicator_code AS debt_indicator, indicator_name, AVG(debt) AS average_debt FROM international_debt GROUP BY debt_indicator, indicator_name ORDER BY average_debt DESC LIMIT 10;")
columns = cursor.description 
result = [{columns[index][0]:column for index, column in enumerate(value)} for value in cursor.fetchall()]
pd.DataFrame(result)

,debt_indicator,indicator_name,average_debt
0,DT.AMT.DLXF.CD,"Principal repayments on external debt, long-te...",5904868401.499193612
1,DT.AMT.DPNG.CD,"Principal repayments on external debt, private...",5161194333.812658349
2,DT.DIS.DLXF.CD,"Disbursements on external debt, long-term (DIS...",2152041216.890243888
3,DT.DIS.OFFT.CD,"PPG, official creditors (DIS, current US$)",1958983452.859836046
4,DT.AMT.PRVT.CD,"PPG, private creditors (AMT, current US$)",1803694101.963265321
5,DT.INT.DLXF.CD,"Interest payments on external debt, long-term ...",1644024067.650806481
6,DT.DIS.BLAT.CD,"PPG, bilateral (DIS, current US$)",1223139290.398230108
7,DT.INT.DPNG.CD,"Interest payments on external debt, private no...",1220410844.421518983
8,DT.AMT.OFFT.CD,"PPG, official creditors (AMT, current US$)",1191187963.083064523
9,DT.AMT.PBND.CD,"PPG, bonds (AMT, current US$)",1082623947.653623188


The indicator DT.AMT.DLXF.CD tops the chart of average debt. This category includes repayment of long term debts. Countries take on long-term debt to acquire immediate capital. Interestingly, there is a huge difference in the amounts of the indicators after the second one. This shows that the first two indicators might be the most severe categories in which the countries owe their debts. 

Next, I will find out which country owes the highest amount of debt in the category of long term debts (DT.AMT.DLXF.CD). This will help understand the county's economic condition a bit more specifically.

In [7]:
# Country that owes the highest amount of debt in category; DT.AMT.DLXF.CD

cursor = conn.cursor()
cursor.execute("SELECT country_name, indicator_name FROM international_debt WHERE debt = (SELECT MAX(debt) FROM international_debt WHERE indicator_code = 'DT.AMT.DLXF.CD');")
columns = cursor.description 
result = [{columns[index][0]:column for index, column in enumerate(value)} for value in cursor.fetchall()]
pd.DataFrame(result)

,country_name,indicator_name
0,China,"Principal repayments on external debt, long-te..."


China appears to be the country with the highest amount of debt in the long-term debt (DT.AMT.DLXF.CD) category. 

Long-term debt is the topmost category when it comes to the average amount of debt. Although, it might not be the most common indicator in which the coutries owe their debt.

In [8]:
# Most common debt indicator

cursor = conn.cursor()
cursor.execute("SELECT indicator_code, COUNT(indicator_code) AS indicator_count FROM international_debt GROUP BY indicator_code ORDER BY indicator_count DESC, indicator_code DESC LIMIT 20;")
columns = cursor.description 
result = [{columns[index][0]:column for index, column in enumerate(value)} for value in cursor.fetchall()]
pd.DataFrame(result)

,indicator_code,indicator_count
0,DT.INT.OFFT.CD,124
1,DT.INT.MLAT.CD,124
2,DT.INT.DLXF.CD,124
3,DT.AMT.OFFT.CD,124
4,DT.AMT.MLAT.CD,124
5,DT.AMT.DLXF.CD,124
6,DT.DIS.DLXF.CD,123
7,DT.INT.BLAT.CD,122
8,DT.DIS.OFFT.CD,122
9,DT.AMT.BLAT.CD,122


There are a total of six debt indicators in which all the countries listed in our dataset have taken debt. DT.AMT.DLXF.CD (highest category with average debt) also appears in the list. Next I will find out the maximum amount of debt each country has. 

In [9]:
cursor = conn.cursor()
cursor.execute("SELECT country_name, MAX(debt) AS maximum_debt FROM international_debt GROUP BY country_name ORDER BY maximum_debt DESC LIMIT 10;")
columns = cursor.description 
result = [{columns[index][0]:column for index, column in enumerate(value)} for value in cursor.fetchall()]
pd.DataFrame(result)

,country_name,maximum_debt
0,China,96218620835.699996948
1,Brazil,90041840304.100006104
2,Russian Federation,66589761833.5
3,Turkey,51555031005.800003052
4,South Asia,48756295898.199996948
5,Least developed countries: UN classification,40160766261.599998474
6,IDA only,34531188113.199996948
7,India,31923507000.799999237
8,Indonesia,30916112653.799999237
9,Kazakhstan,27482093686.400001526


China happens to have the highest maximum debt in the table. 

In this project, I have successfully looked at debt owed by countries across the globe. I have extracted a few summary statistics from the data and unraveled some interesting facts and figures.